# 2021/04/15 - 1

## 必要なモジュールのインポート

In [ ]:
# まずは必要になるpythonモジュールをすべてインポートしておく
import numpy as np
import matplotlib.pyplot as plt
from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit, IBMQ, Aer, execute, transpile, assemble
from qiskit.providers.ibmq import least_busy
from qiskit.tools.monitor import job_monitor
from qiskit.visualization import plot_histogram

# utilsはこのワークブック独自のモジュール（インポートエラーが出る場合はPYTHONPATHを設定するか、sys.pathをいじってください）
from utils.show_state import show_state
from utils.optimized_additions import optimized_additions, get_initial_layout

### IBMQ認証（一度だけ）

[IBM Quantum Experience](https://quantum-computing.ibm.com/)のホームページからトークンをコピーしてきてください。

In [ ]:
IBMQ.save_account('__paste_your_token_here__')

### アカウントのアクティベート（毎回）

In [ ]:
IBMQ.load_account()

## CHSH不等式を計算する回路を書く

4つの回路でそれぞれ「ベル状態」

$$
\frac{1}{\sqrt{2}} \left( \ket{00} + \ket{11} \right)
$$

を作ったあと、別々の$R_y$ゲートをかけます。

In [ ]:
circuits = []
for ic in range(4):
    circuit = QuantumCircuit(2, name='circuit{}'.format(ic))
    circuit.h(0)
    circuit.cx(0, 1)
    circuits.append(circuit)

circuits[0].ry(-np.pi / 4., 1)
circuits[1].ry(-3. * np.pi / 4., 1)
circuits[2].ry(-np.pi / 4., 1)
circuits[3].ry(-3. * np.pi / 4., 1)

circuits[2].ry(-np.pi / 2., 0)
circuits[3].ry(-np.pi / 2., 0)

for circuit in circuits:
    circuit.measure_all()

# draw()にmatplotlibのaxesオブジェクトを渡すと、そこに描画してくれる
# 一つのノートブックセルで複数プロットしたい時などに便利
for circuit in circuits:
    ax = plt.figure().add_subplot()
    circuit.draw('mpl', ax=ax)

## 回路をシミュレータで実行する

ワークブックでは直接回路を実機に渡していますが、講義では時間が限られているので、まずシミュレータを利用します。

### ジョブの実行

In [ ]:
simulator = Aer.get_backend('qasm_simulator')
print(simulator.name())

shots = 10000
    
sim_job = execute(circuits, backend=simulator, shots=shots)

sim_result = sim_job.result()

### 測定結果の解析 - 個々の回路のヒストグラム

In [ ]:
sim_counts = []
for circuit in circuits:
    c = sim_result.get_counts(circuit)
    sim_counts.append(c)
    
for c in sim_counts:
    ax = plt.figure().add_subplot()
    plot_histogram(c, ax=ax)

### 測定結果の解析 - CHSH不等式の$S$

In [ ]:
C = []
for c in sim_counts:
    C.append((c['00'] + c['11'] - c['01'] - c['10']) / shots)
    
S = C[0] - C[1] + C[2] + C[3]
print('S =', S)

## 回路を実機で実行する

### バックエンドの決定

In [ ]:
# IBMQプロバイダ（実機へのアクセスを管理するオブジェクト）
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')

# バックエンド（実機）のうち量子ビット数2個以上のもののリストをプロバイダから取得し、一番空いているものを選ぶ
backend_filter = lambda b: (not b.configuration().simulator) and (b.configuration().n_qubits >= 2) and b.status().operational
backend = least_busy(provider.backends(filters=backend_filter))

print('Jobs will run on', backend.name())

### ジョブの実行

In [ ]:
shots = 8192

job = execute(circuits, backend=backend, shots=shots)

job_monitor(job, interval=2)

result = job.result()

### ヒストグラム

In [ ]:
counts = []
for circuit in circuits:
    c = result.get_counts(circuit)
    counts.append(c)
    
for c in counts:
    ax = plt.figure().add_subplot()
    plot_histogram(c, ax=ax)

In [ ]:
C = []
for c in counts:
    C.append((c['00'] + c['11'] - c['01'] - c['10']) / shots)
    
S = C[0] - C[1] + C[2] + C[3]

print('C:', C)
print('S =', S)
if S > 2.:
    print('Yes, we are using a quantum computer!')